In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Flatten, Dense, Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tabulate import tabulate

In [2]:
# Baca dataset CSV
df = pd.read_csv('/content/Saiko-Dataset.csv')

In [3]:
df

,id,Kecamatan,Kelurahan,Komuditas,Deskripsi,Harga
0,1101.0,Abeli,Abeli,Ikan Bandeng,"Ikan bandeng segar, pilihan terbaik untuk berb...","30.000,00"
1,1102.0,Abeli,Abeli,Ikan Lele,"Ikan Lele segar, siap menjadi hidangan favorit...","23.000,00"
2,1103.0,Abeli,Abeli,Ikan Betok,"Ikan betok segar, pilihan tepat untuk hidangan...","30.000,00"
3,1104.0,Abeli,Abeli,Belut Air Tawar,"Belut air tawar segar, siap diolah menjadi ber...","70.000,00"
4,1105.0,Abeli,Abeli,Ikan Emas,"Ikan emas segar, langsung dari kolam ke dapur ...","44.000,00"
...,...,...,...,...,...,...
2031,NaN,NaN,Silver Carp,Ikan Mas Perak,NaN,NaN
2032,NaN,NaN,Silver Perch,Bertengger Perak,NaN,NaN
2033,NaN,NaN,Snakehead,Kepala ular,NaN,NaN
2034,NaN,NaN,Tenpounder,Sepuluh pon,NaN,NaN


In [4]:
df.describe()

,id
count,842.000000
mean,3446.345606
std,1592.984844
min,1101.000000
25%,2101.250000
50%,3401.500000
75%,5201.750000
max,5902.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2036 entries, 0 to 2035
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         842 non-null    float64
 1   Kecamatan  1950 non-null   object 
 2   Kelurahan  1981 non-null   object 
 3   Komuditas  1981 non-null   object 
 4   Deskripsi  1965 non-null   object 
 5   Harga      1945 non-null   object 
dtypes: float64(1), object(5)
memory usage: 95.6+ KB


In [6]:
df.isnull().sum()

id           1194
Kecamatan      86
Kelurahan      55
Komuditas      55
Deskripsi      71
Harga          91
dtype: int64

In [7]:
df.columns

Index(['id', 'Kecamatan', 'Kelurahan', 'Komuditas', 'Deskripsi', 'Harga'], dtype='object')

In [10]:
# Hapus baris yang memiliki nilai NaN
df = df.dropna()

In [11]:
df

,id,Kecamatan,Kelurahan,Komuditas,Deskripsi,Harga
0,1101.0,Abeli,Abeli,Ikan Bandeng,"Ikan bandeng segar, pilihan terbaik untuk berb...","30.000,00"
1,1102.0,Abeli,Abeli,Ikan Lele,"Ikan Lele segar, siap menjadi hidangan favorit...","23.000,00"
2,1103.0,Abeli,Abeli,Ikan Betok,"Ikan betok segar, pilihan tepat untuk hidangan...","30.000,00"
3,1104.0,Abeli,Abeli,Belut Air Tawar,"Belut air tawar segar, siap diolah menjadi ber...","70.000,00"
4,1105.0,Abeli,Abeli,Ikan Emas,"Ikan emas segar, langsung dari kolam ke dapur ...","44.000,00"
...,...,...,...,...,...,...
837,5828.0,Kendari,Mata,Labu,"Labu berkualitas premium, dijual per buah seha...","10.600,00"
838,5829.0,Kendari,Mata,Lobak,Harap dibaca deskripsi sebelum melakukan pemes...,"6.900,00"
839,5830.0,Kendari,Mata,Tomat,"Tomat merah dan segar, dijual per buah dengan ...","1.500,00"
840,5901.0,Kendari,Purirano,Ikan Bandeng,"Ikan bandeng segar, kaya akan nutrisi untuk me...","31.000,00"


In [12]:
# Gabungkan kolom Deskripsi dan Komoditas
data['tags'] = data['Deskripsi'] + " " + data['Komuditas']

<ipython-input-12-16956f230ad4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tags'] = data['Deskripsi'] + " " + data['Komuditas']


In [14]:
data

,id,Kecamatan,Kelurahan,Komuditas,Deskripsi,Harga,tags
0,1101.0,Abeli,Abeli,Ikan Bandeng,"Ikan bandeng segar, pilihan terbaik untuk berb...","30.000,00","Ikan bandeng segar, pilihan terbaik untuk berb..."
1,1102.0,Abeli,Abeli,Ikan Lele,"Ikan Lele segar, siap menjadi hidangan favorit...","23.000,00","Ikan Lele segar, siap menjadi hidangan favorit..."
2,1103.0,Abeli,Abeli,Ikan Betok,"Ikan betok segar, pilihan tepat untuk hidangan...","30.000,00","Ikan betok segar, pilihan tepat untuk hidangan..."
3,1104.0,Abeli,Abeli,Belut Air Tawar,"Belut air tawar segar, siap diolah menjadi ber...","70.000,00","Belut air tawar segar, siap diolah menjadi ber..."
4,1105.0,Abeli,Abeli,Ikan Emas,"Ikan emas segar, langsung dari kolam ke dapur ...","44.000,00","Ikan emas segar, langsung dari kolam ke dapur ..."
...,...,...,...,...,...,...,...
837,5828.0,Kendari,Mata,Labu,"Labu berkualitas premium, dijual per buah seha...","10.600,00","Labu berkualitas premium, dijual per buah seha..."
838,5829.0,Kendari,Mata,Lobak,Harap dibaca deskripsi sebelum melakukan pemes...,"6.900,00",Harap dibaca deskripsi sebelum melakukan pemes...
839,5830.0,Kendari,Mata,Tomat,"Tomat merah dan segar, dijual per buah dengan ...","1.500,00","Tomat merah dan segar, dijual per buah dengan ..."
840,5901.0,Kendari,Purirano,Ikan Bandeng,"Ikan bandeng segar, kaya akan nutrisi untuk me...","31.000,00","Ikan bandeng segar, kaya akan nutrisi untuk me..."


In [15]:
data  = data.drop(columns=['Deskripsi'])

In [16]:
data

,id,Kecamatan,Kelurahan,Komuditas,Harga,tags
0,1101.0,Abeli,Abeli,Ikan Bandeng,"30.000,00","Ikan bandeng segar, pilihan terbaik untuk berb..."
1,1102.0,Abeli,Abeli,Ikan Lele,"23.000,00","Ikan Lele segar, siap menjadi hidangan favorit..."
2,1103.0,Abeli,Abeli,Ikan Betok,"30.000,00","Ikan betok segar, pilihan tepat untuk hidangan..."
3,1104.0,Abeli,Abeli,Belut Air Tawar,"70.000,00","Belut air tawar segar, siap diolah menjadi ber..."
4,1105.0,Abeli,Abeli,Ikan Emas,"44.000,00","Ikan emas segar, langsung dari kolam ke dapur ..."
...,...,...,...,...,...,...
837,5828.0,Kendari,Mata,Labu,"10.600,00","Labu berkualitas premium, dijual per buah seha..."
838,5829.0,Kendari,Mata,Lobak,"6.900,00",Harap dibaca deskripsi sebelum melakukan pemes...
839,5830.0,Kendari,Mata,Tomat,"1.500,00","Tomat merah dan segar, dijual per buah dengan ..."
840,5901.0,Kendari,Purirano,Ikan Bandeng,"31.000,00","Ikan bandeng segar, kaya akan nutrisi untuk me..."


In [21]:
# Tokenisasi dan padding untuk kolom tags
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['tags'])

# Maksimal panjang sequence
max_length = 100

# Konversi teks menjadi sequence
sequences = tokenizer.texts_to_sequences(data['tags'])
X = pad_sequences(sequences, maxlen=max_length)

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data['Komuditas'])

# Pisahkan data menjadi set pelatihan dan set pengujian
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [22]:
# Buat model
vocab_size = len(tokenizer.word_index) + 1  # +1 untuk padding

input_text = Input(shape=(max_length,), name='text_input')
embedding = Embedding(input_dim=vocab_size, output_dim=50, input_length=max_length)(input_text)
flatten = Flatten()(embedding)

# Tambahkan layer dense
dense = Dense(64, activation='relu')(flatten)
output = Dense(len(label_encoder.classes_), activation='softmax')(dense)  # Ubah output sesuai jumlah komoditas

# Buat model
model = Model(inputs=input_text, outputs=output)

# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Latih model
history = model.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test))

Epoch 1/10
21/21 [==============================] - 1s 19ms/step - loss: 3.4255 - accuracy: 0.0402 - val_loss: 3.3645 - val_accuracy: 0.0774
Epoch 2/10
21/21 [==============================] - 0s 9ms/step - loss: 3.2631 - accuracy: 0.1714 - val_loss: 3.1911 - val_accuracy: 0.1726
Epoch 3/10
21/21 [==============================] - 0s 10ms/step - loss: 3.0017 - accuracy: 0.1967 - val_loss: 2.8696 - val_accuracy: 0.1786
Epoch 4/10
21/21 [==============================] - 0s 11ms/step - loss: 2.5992 - accuracy: 0.2846 - val_loss: 2.4744 - val_accuracy: 0.2857
Epoch 5/10
21/21 [==============================] - 0s 9ms/step - loss: 2.1406 - accuracy: 0.5022 - val_loss: 1.9992 - val_accuracy: 0.5833
Epoch 6/10
21/21 [==============================] - 0s 12ms/step - loss: 1.7028 - accuracy: 0.6990 - val_loss: 1.6482 - val_accuracy: 0.6905
Epoch 7/10
21/21 [==============================] - 0s 9ms/step - loss: 1.3232 - accuracy: 0.8092 - val_loss: 1.2885 - val_accuracy: 0.7679
Epoch 8/10
21/21

In [23]:
# Evaluasi model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')

6/6 [==============================] - 0s 3ms/step - loss: 0.5952 - accuracy: 0.9286
Test accuracy: 0.9285714030265808


In [33]:
# Fungsi pencarian komoditas berdasarkan query
def search_commodities(search_query, data):
    # Tampilkan semua baris yang cocok dengan query pencarian di kolom tags
    matched_rows = data[data['tags'].str.contains(search_query, case=False, na=False)]
    return matched_rows[['Kecamatan', 'Kelurahan', 'Komuditas', 'Harga', 'tags']]

# Contoh penggunaan fungsi pencarian
search_query = "wortel segar dan renyah"

# Tampilkan semua baris yang cocok dengan query pencarian untuk data prediksi/pengujian
matched_rows = search_commodities(search_query, data)
print(f'Matched Rows for "{search_query}":')
print(tabulate(matched_rows, headers='keys', tablefmt='psql'))

Matched Rows for "wortel segar dan renyah":
+-----+-------------+-------------+-------------+-----------+-----------------------------------------------------------------------------------+
|     | Kecamatan   | Kelurahan   | Komuditas   | Harga     | tags                                                                              |
|-----+-------------+-------------+-------------+-----------+-----------------------------------------------------------------------------------|
|  22 | Abeli       | Abeli       | Wortel      | 10.000,00 | Wortel segar dan renyah, penuh dengan vitamin A untuk kesehatan mata Anda! Wortel |
| 412 | Kadia       | Kadia       | Wortel      | 10.000,00 | Wortel segar dan renyah, penuh dengan vitamin A untuk kesehatan mata Anda! Wortel |
+-----+-------------+-------------+-------------+-----------+-----------------------------------------------------------------------------------+


In [39]:
from tensorflow.keras.models import save_model

# Simpan model dalam format H5
save_model(model, 'recommendation_model.h5')

<ipython-input-39-ed584eb2112c>:4: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model, 'recommendation_model.h5')


In [42]:
# Load model yang disimpan dalam format H5
saved_model_path = 'recommendation_model.h5'
model = tf.keras.models.load_model(saved_model_path)

# Konverter TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan model TensorFlow Lite
with open('recommendation_model.tflite', 'wb') as f:
    f.write(tflite_model)